In [1]:
import pickle
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np

%matplotlib inline

### Helper Functions

The first two are for loading and storing dictionaries to/from a pickle file

In [4]:
def store_dict_pickle(file_name, dictionary):
    """
        Saves a dictionary to a pickle file

        :param file_name: the name of the file the data will be stored in
        :type file_name: str

        :param dictionary: the dictionary to be stored in the file
        :type dictionary: dict
    """

    with open('data/' + file_name + '.pkl', 'wb') as f:
        pickle.dump(dictionary, f, pickle.HIGHEST_PROTOCOL)


def load_dict_pickle(file_name):
    """
    Retrieves a dictionary form the specified file

    :param file_name: the name of the file to load
    :type file_name: str

    :return: the dictionary retrieved from the file
    :rtype dict
    """

    with open('data/' + file_name + '.pkl', 'rb') as f:
        dictionary =  pickle.load(f)
    return dictionary

# Loading Artist-Country Data

In [5]:
#intializing dictionary variables
dict_05 = load_dict_pickle("2005_country_artist")
dict_06 = load_dict_pickle("2006_country_artist")
dict_07 = load_dict_pickle("2007_country_artist")
dict_08 = load_dict_pickle("2008_country_artist")
dict_09 = load_dict_pickle("2009_country_artist")
dict_10 = load_dict_pickle("2010_country_artist")
dict_13 = load_dict_pickle("2013_country_artist")

# Initializing Graphs From Dictionaries

In [68]:
import copy

def top_artists(year_dict, top_number):
    # create graph to represent trends for given year
    G = nx.Graph()
    
    # loop through each artist in the dictionary
    for country in year_dict:
        # create country node & add it
        G.add_node(country)
        
        # get top 5 artists for current country
        artists = copy.copy(year_dict[country])
        num_artist = len(artists)
        for i in range(top_number):
            # make sure the country has enough top artists
            if( num_artist < top_number):
                # remove country's node and skip it
                G.remove_node(country)
                break;
            
            # find ith top artist
            top = max(artists.keys(), key=(lambda k: artists[k]))
            
            #create node for ith top artist and make a conenction b/n it and the country
            G.add_node(top)
            G.add_edge(country, top, weight=year_dict[country][top])
            
            artists.pop(top)

    return G

In [81]:
G = top_artists(dict_08, 10)
# pos = nx.spring_layout(G)
# nx.draw(G, pos=pos);
# plt.axis('off')
# plt.show()